# Computational Economics  (ECO288)

## Julia Basics

### What is Julia

- developped at MIT on top of opensource technologies
    - linux / git / llvm
- syntax inspired by Matlab but:
    - more consistent
    - lots of features from high level languages
- everything is JIT-compiled
    - interpreted vs compiled treadeoff
    - -> very fast
    - most of the base library is written in Julia
- opensource/free + vibrant community


Some useful links from QuantEcon:

* [Julia cheatsheet](https://cheatsheets.quantecon.org/julia-cheatsheet.html)
* [Julia-Matlab comparison](https://cheatsheets.quantecon.org/index.html)
* [Julia essentials](https://lectures.quantecon.org/jl/julia_essentials.html)
* [Vectors, arrays and matrices](https://lectures.quantecon.org/jl/julia_arrays.html)

Excellent resources at: [julialang](https://julialang.org/learning/)
- checkout JuliaAcademy, it's free

### an example of what you shouldn't do in Matlab

How I learnt: interpreted code is slow, so vectorize your coe.

In [ ]:
function stupid_loop(I,J,K)
    t = 0.0
    for i=1:I
        for j=1:J
            for k = 1:K
                t += 1.0
            end        
        end
    end
    return t
end
@time [ stupid_loop(1000,1000,i) for i =1:10]

Code is translated to LLVM code then to instructions for the processor. Note that processor instructions are shorter than LLVM code.

In [ ]:
@code_llvm stupid_loop(10,10,10)

In [ ]:
@code_native stupid_loop(10,10,10)

### Syntax Review

#### Variable assignment

Assignement operator is = (equality is ==, identity is ===)

In [ ]:
# Assign the value 10 to the variable x
x = 10

In [ ]:
2 == 3

In [ ]:
# Variable names can have Unicode characters
# To get ϵ in the REPL, type \epsilon<TAB>
ϵ = 1e-4

Default semantic is pass-by-reference:

In [ ]:
a = [1,2,3,4]
b = a
a[1] = 10
b

To work on a copy: `copy` or `deepcopy`

In [ ]:
a = [1,2,3,4]
b = copy(a)
a[1]=10
b

In [ ]:
a .== b

In [ ]:
a === b

#### Basic types

In [ ]:
# for any object `typeof` returns the type
?typeof

In [ ]:
typeof(a)

#### Numbers

In [ ]:
y = 2 + 2

In [ ]:
-y

In [ ]:
0.34*23

In [ ]:
3/4

In [ ]:
# Scalar multiplication doesn't require *
3(4 - 2)

In [ ]:
x = 4
2x

In [ ]:
typeof(x)

In [ ]:
sizeof(a)

#### Booleans

Equality

In [ ]:
0 == 1

In [ ]:
2 != 3

In [ ]:
3 <= 4

Identity

In [ ]:
a = [34, 35]
b = [34, 35]
c = a

In [ ]:
c === a

In [ ]:
b === a

Boolean operator

In [ ]:
true && false

In [ ]:
true || false

In [ ]:
!true

#### Strings

In [ ]:
# Strings are written using double quotes
str = "This is a string"

In [ ]:
ch = 'k' # this is a character

In [ ]:
# Strings can also contain Unicode characters
fancy_str = "α is a string"

In [ ]:
# String interpolation using $
# The expression in parentheses is evaluated and the result is 
# inserted into the string
a = 2+2
"2 + 2 = $(a)"

In [ ]:
println("It took me $(a) iterations")

In [ ]:
# String concatenation using *
"hello" * "world"

In [ ]:
println("hello ", "world")

#### Arrays

Julia has one-dimensional arrays. They are also called Vector.

In [ ]:
A = [1, 2]

In [ ]:
typeof(A) == Vector{Int64}

In [ ]:
A''

2d arrays are also called matrices... and can be used for matrix multiplications.

In [ ]:
a1 = [1,2,3,4]
a2 = [1,2,3,4].+4
[a1; a2]
cat(a1, a2; dims=1)

In [ ]:
b = [1 0.6 0]

In [ ]:
B = [0.1 0.2 0.3; 4 5 6]

In [ ]:
B*B'

Vectorized operations take a ., even comparisons:

In [ ]:
B.*B .< B

In [ ]:
f(x) = x^2+1

In [ ]:
f(43)

In [ ]:
f.(B)

Elements are always accessed with square brackets:

In [ ]:
B[:,1]

In [ ]:
B[:,1:end-1]

#### Control flow

Conditions

In [ ]:
x = -3
if x < 0
    println("x is negative")
elseif x > 0 # optional and unlimited
    println("x is positive")
else         # optional
    println("x is zero")
end

While

In [ ]:
i = 3
while i > 0
    println(i)
    i = i - 1
end

For loops

In [ ]:
# Iterate through ranges of numbers
for i = 1:3
    println(i)
end

In [ ]:
# Iterate through arrays
cities = ["Boston", "New York", "Philadelphia"]
for city in cities
    println(city)
end

In [ ]:
cities

In [ ]:
states

In [ ]:
collect( zip(cities, states) )

In [ ]:
# Iterate through arrays of tuples using zip
states = ["MA", "NY", "PA"]
for (city, state) in zip(cities, states)
    println("$city, $state")
end

In [ ]:
# Iterate through arrays and their indices using enumerate
for (i, city) in enumerate(cities)
    println("City $i is $city")
end

#### List comprehensions

In [ ]:
[i^2 for i=1:10]

In [ ]:
[i^2 for i=1:10 if mod(i,2)==0]

### Data Types and multiple dispatch

#### Composite types

A **composite type** is a collection of named fields that can be treated as a single value. They bear a passing resemblance to MATLAB structs.

All fields must be declared ahead of time. The double colon, `::`, constrains a field to contain values of a certain type. This is optional for any field.

In [ ]:
# Type definition
struct Parameter
    value::Float64
    transformation::Function # Function is a type!
    tex_label::String
    description::String
end

When a type with $n$ fields is defined, a constructor (function that creates an instance of that type) that takes $n$ ordered arguments is automatically created. Additional constructors can be defined for convenience.

In [ ]:
# Creating an instance of the Parameter type using the default
# constructor
β = Parameter(0.9, identity, "\beta", "Discount rate")

In [ ]:
# Alternative constructors end with an appeal to the default
# constructor
function Parameter(value::Float64, tex_label::String)
    transformation = identity
    description = "No description available"
    return Parameter(value, transformation, tex_label, description)
end

α = Parameter(0.5, "\alpha")

Now the function `Parameter` has two different `methods` with different signatures:

In [ ]:
methods(Parameter)

In [ ]:
fun(x::Int64, y::Int64) = x^3 + y

In [ ]:
fun(x::Float64, y::Int64) = x/2 + y

In [ ]:
fun(2, 2)

In [ ]:
fun(2.0, 2)

In [ ]:
# Find the fields of an instance of a composite type
fieldnames(α)

In [ ]:
α.tex_label

In [ ]:
# Access a particular field using .
α.value

In [ ]:
# Fields are modifiable and can be assigned to, like 
# ordinary variables
α.value = 0.75

### Parameterized Types

**Parameterized types** are data types that are defined to handle values identically regardless of the type of those values.

Arrays are a familiar example. An `Array{T,1}` is a one-dimensional array filled with objects of any type `T` (e.g. `Float64`, `String`).

In [ ]:
# Defining a parametric point
struct Duple{T} # T is a parameter to the type Duple
    x::T
    y::T
end

This single declaration defines an unlimited number of new types: `Duple{String}`, `Duple{Float64}`, etc. are all immediately usable.

In [ ]:
sizeof( Duple(3.0, -15.0) )

In [ ]:
Duple("Broadway", "42nd St")

In [ ]:
# What happens here?
Duple(1.5, 3)

We can also restrict the type parameter `T`:

In [ ]:
# T can be any subtype of Number, but nothing else
struct PlanarCoordinate{T<:Number}
    x::T
    y::T
end

In [ ]:
PlanarCoordinate("4th Ave", "14th St")

In [ ]:
PlanarCoordinate(2//3, 8//9)

### Why Use Types?

You can write all your code without thinking about types at all. If you do this, however, you’ll be missing out on some of the biggest benefits of using Julia.

If you understand types, you can:

- Write faster code
- Write expressive, clear, and well-structured programs (keep this in mind when we talk about functions)
- Reason more clearly about how your code works

Even if you only use built-in functions and types, your code still takes advantage of Julia’s type system. That’s why it’s important to understand what types are and how to use them.

In [ ]:
# Example: writing type-stable functions
function sumofsins_unstable(n::Integer)  
    sum = 0  
    for i in 1:n  
        sum += sin(3.4)  
    end  
    return sum 
end  

function sumofsins_stable(n::Integer)  
    sum = 0.0  
    for i in 1:n  
        sum += sin(3.4)  
    end  
    return sum 
end

# Compile and run
sumofsins_unstable(Int(1e5))
sumofsins_stable(Int(1e5))

In [ ]:
@time sumofsins_unstable(Int(1e5))

In [ ]:
@time sumofsins_stable(Int(1e5))

In `sumofsins_stable`, the compiler is guaranteed that `sum` is of type `Float64` throughout; therefore, it saves time and memory. On the other hand, in `sumofsins_unstable`, the compiler must check the type of `sum` at each iteration of the loop. Let's look at the LLVM [intermediate representation](http://www.johnmyleswhite.com/notebook/2013/12/06/writing-type-stable-code-in-julia/).

### Multiple Dispatch

So far we have defined functions over argument lists of any type. Methods allow us to define functions “piecewise”. For any set of input arguments, we can define a **method**, a definition of one possible behavior for a function.

In [ ]:
# Define one method of the function print_type
function print_type(x::Number)
    println("$x is a number")
end

In [ ]:
# Define another method
function print_type(x::String)
    println("$x is a string")
end

In [ ]:
# Define yet another method
function print_type(x::Number, y::Number)
    println("$x and $y are both numbers")
end

In [ ]:
# See all methods for a given function
methods(print_type)

Julia uses **multiple dispatch** to decide which method of a function to execute when a function is applied. In particular, Julia compares the types of _all_ arguments to the signatures of the function’s methods in order to choose the applicable one, not just the first (hence "multiple").

In [ ]:
print_type(5)

In [ ]:
print_type("foo")

In [ ]:
print_type([1, 2, 3])

#### Other types of functions

Julia supports a short function definition for one-liners

In [ ]:
f(x::Float64) = x^2.0
f(x::Int64) = x^3

As well as a special syntax for anonymous functions

In [ ]:
u->u^2

In [ ]:
map(u->u^2, [1,2,3,4])

### Writing Julian Code

As we've seen, you can use Julia just like you use MATLAB and get faster code. However, to write faster and _better_ code, attempt to write in a “Julian” manner:

- Define composite types as logically needed
- Write type-stable functions for best performance
- Take advantage of multiple dispatch to write code that looks like math
- Add methods to existing functions

### Just-in-Time Compilation

How is Julia so fast? Julia is just-in-time (JIT) compiled, which  means (according to [this StackExchange answer](http://stackoverflow.com/questions/95635/what-does-a-just-in-time-jit-compiler-do)):

> A JIT compiler runs after the program has started and compiles the code (usually bytecode or some kind of VM instructions) on the fly (or just-in-time, as it's called) into a form that's usually faster, typically the host CPU's native instruction set. _A JIT has access to dynamic runtime information whereas a standard compiler doesn't and can make better optimizations like inlining functions that are used frequently._

> This is in contrast to a traditional compiler that compiles all the code to machine language before the program is first run.

In particular, Julia uses type information at runtime to optimize how your code is compiled. This is why writing type-stable code makes such a difference in speed!

## Additional Exercises

Taken from QuantEcon's [Julia Essentials](https://lectures.quantecon.org/jl/julia_essentials.html) and [Vectors, Arrays, and Matrices](https://lectures.quantecon.org/jl/julia_arrays.html) lectures.

1. Consider the polynomial $$p(x) = \sum_{i=0}^n a_0 x^0$$ Using `enumerate`, write a function `p` such that `p(x, coeff)` computes the value of the polynomial with coefficients `coeff` evaluated at `x`.

2. Write a function `solve_discrete_lyapunov` that solves the discrete Lyapunov equation $$S = ASA' + \Sigma \Sigma'$$ using the iterative procedure $$S_0 = \Sigma \Sigma'$$ $$S_{t+1} = A S_t A' + \Sigma \Sigma'$$ taking in as arguments the $n \times n$ matrix $A$, the $n \times k$ matrix $\Sigma$, and a number of iterations.